In [1]:
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

# Set hyper-params

In [2]:
input_size = 28*28
num_classes = 10
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# CIFAR-10 dataset

In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [3]:
train_dataset = torchvision.datasets.MNIST(root='./', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Create logistic regression model

In [4]:
model = nn.Linear(input_size, num_classes)

# Loss and optimizer

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train model

In [7]:
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # reshape image to (batch_size, input_size)
    images = images.reshape(-1, input_size)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {}' 
              .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/600], Loss: 2.20538067817688
Epoch [1/10], Step [200/600], Loss: 2.150702714920044
Epoch [1/10], Step [300/600], Loss: 2.0327565670013428
Epoch [1/10], Step [400/600], Loss: 1.9329712390899658
Epoch [1/10], Step [500/600], Loss: 1.8697079420089722
Epoch [1/10], Step [600/600], Loss: 1.827599287033081
Epoch [2/10], Step [100/600], Loss: 1.7115004062652588
Epoch [2/10], Step [200/600], Loss: 1.6478251218795776
Epoch [2/10], Step [300/600], Loss: 1.6571251153945923
Epoch [2/10], Step [400/600], Loss: 1.5286588668823242
Epoch [2/10], Step [500/600], Loss: 1.5100370645523071
Epoch [2/10], Step [600/600], Loss: 1.5477768182754517
Epoch [3/10], Step [100/600], Loss: 1.3350709676742554
Epoch [3/10], Step [200/600], Loss: 1.3477046489715576
Epoch [3/10], Step [300/600], Loss: 1.3414307832717896
Epoch [3/10], Step [400/600], Loss: 1.3184930086135864
Epoch [3/10], Step [500/600], Loss: 1.3043997287750244
Epoch [3/10], Step [600/600], Loss: 1.2914760112762451
Epoch [4/10], 

# Test model

In [8]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels, in test_loader:
    images = images.reshape(-1, input_size)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

  print("Acc: ", (100*correct)/total)

Acc:  tensor(85.2600)


# Save model

In [9]:
torch.save(model.state_dict(), 'logistic_regression_model.ckpt')